## Scholix

### Pass the dataset DOIs to the scholex API to get the citations and their respective DOIs

In [ ]:
import requests, time, json, re, datetime, os, sys
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import numpy as np
import pandas as pd
from math import ceil
from datetime import date
from scholix_fun import getNERCDataDOIs, getScholixDatasetCitations, process_citation_results, getPublicationType, countScholixCitations, getCitationString
import crossRef_fun

# import exportCitationResultsToCsv
# from Results import convertCSVtoJSON


sys.path.insert(0, '..')

In [1]:
# this takes about 8 mins
scholex_df = getScholixDatasetCitations.getScholixDatasetCitations(dataCite_df)

NameError: name 'getScholixDatasetCitations' is not defined

#### Process the citation results

In [ ]:
dataCite_df, scholex_df = process_citation_results.process_citation_results(dataCite_df, scholex_df)

In [ ]:
# filter out gbif registrant code prefix 10.15468
# scholex_df = scholex_df[~scholex_df.pubID.str.contains("10.15468")]
scholex_df = scholex_df[~scholex_df['pubID'].apply(lambda x: str(x)).str.contains("10.15468")]


### Check the DOIs at DOI.org to determine the type of publication and to check there are no duplicates (by preprints etc) 

In [ ]:
# very long 3+ hours
scholex_df = getPublicationType.getPublicationType(scholex_df)

### Output spreadsheet

In [ ]:
today = date.today()

scholex_filename = "Results/Intermediate data/" + 'scholix_citation_publication_info_' + (today.strftime("%d%m%Y")) + '.csv'
scholex_df.to_csv(scholex_filename, index = False)
scholex_df.to_csv("Results/Intermediate data/latest_results_scholix.csv", index = False)